# Predicting Delays Based on Flight Data

### Import Statements

In [2]:
# make sure to use the same kernel as the one you use for class

import pandas as pd
import numpy as np

### Import Data

In [3]:
data = pd.read_csv('Airlines.csv')

print(data.head()) #check if data imported correctly

   id Airline  Flight AirportFrom AirportTo  DayOfWeek  Time  Length  Delay
0   1      CO     269         SFO       IAH          3    15     205      1
1   2      US    1558         PHX       CLT          3    15     222      1
2   3      AA    2400         LAX       DFW          3    20     165      1
3   4      AA    2466         SFO       DFW          3    20     195      1
4   5      AS     108         ANC       SEA          3    30     202      0
